In [ ]:
%pip install ms-fabric-cli --quiet

# Variables

## First Time Setup
If it´s the first time running the script, set the parameter 'FIRST_RUN' as 'True', else set it up as 'False'

In [ ]:
FIRST_RUN = True

## Environemnt Variables

- Add the target Workspace in workspace_name. It will use the target Workspace or create it.
- Add the Capacity name in capacity_name. It will be used only if the workspace doesn´t exist and for the capacity assignment to that workspace.
- Add the target Eventhouse name in eventhouse_name. Can be an existing Eventhouse. If blank will use detault name.

In [ ]:
workspace_name = "[Dev] Microsoft Fabric Platform Monitoring"
capacity_name="" #CAPACITY NAME
eventhouse_name = "Fabric Platform Monitoring"

## Capacity

In [ ]:
# Specify if you want to install the Capacity Module
INSTALL_CAPACITY_MODULE = False

# Capacity ID of the capacity you want to setup. You can change it manually later in the Eventstream, 
# or add more capacities
capacity_id = f""

## Gateway

In [ ]:
# Specify if you want to install the Gateway Module
INSTALL_GATEWAY_MODULE = False

## Audit and Inventory

In [ ]:
# Specify if you want to install the Platform Activity Events and Inventory Module
# If the inventory module is used, the activity events is recommended
INSTALL_ACTIVITY_EVENTS_MODULE = True
INSTALL_INVENTORY_MODULE = True

### For the Platform Monitoring Module, 
### you need to setup an Azure Key Vault to store the SPN with the Admin API Access,
### and Admin access to the gateways to list. The SPN needs to be in a Servurity Group with
### the Tenant settings access to Admin and Regular Fabric API.
###
### IMPORTANT: Add the Security group also to this workspace

# Key Vault URI
key_vault_uri = f""

# Key Vault secret name with the tenant id
key_vault_tenant_id = f""

# Key vault secret name with the App Id of the Service Principal
key_vault_client_id = f""

# Key vault secret name with the secret of the Service Principal
key_vault_client_secret = f""

## Source Git Repo

In [ ]:
##### DO NET CHANGE UNLESS SPECIFIED OTHERWISE ####
repo_owner = "ecotte"
repo_name = "Fabric-Monitoring-RTI"
branch = "Capacity"
folder_prefix = ""
###################################################

# Process

## Load Libraries

In [ ]:
import subprocess
import os
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import sempy.fabric as fabric

## Download of source & config files
This part downloads all source and config files needed for the deployment into the ressources of the notebook

In [ ]:
def download_folder_as_zip(repo_owner, repo_name, output_zip, branch="main", folder_to_extract="src",  remove_folder_prefix = ""):
    # Construct the URL for the GitHub API to download the repository as a zip file
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"
    
    # Make a request to the GitHub API
    response = requests.get(url)
    response.raise_for_status()
    
    # Ensure the directory for the output zip file exists
    os.makedirs(os.path.dirname(output_zip), exist_ok=True)
    
    # Create a zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zipf:
        with zipfile.ZipFile(output_zip, 'w') as output_zipf:
            for file_info in zipf.infolist():
                parts = file_info.filename.split('/')
                if  re.sub(r'^.*?/', '/', file_info.filename).startswith(folder_to_extract): 
                    # Extract only the specified folder
                    file_data = zipf.read(file_info.filename)  
                    if folder_prefix != "":
                        parts.remove(remove_folder_prefix)
                    output_zipf.writestr(('/'.join(parts[1:])), file_data)
def uncompress_zip_to_folder(zip_path, extract_to):
    # Ensure the directory for extraction exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Uncompress all files from the zip into the specified folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Delete the original zip file
    os.remove(zip_path)

download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/src/src.zip", branch = branch, folder_to_extract= f"{folder_prefix}/src", remove_folder_prefix = f"{folder_prefix}")
download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/config/config.zip", branch = branch, folder_to_extract= f"{folder_prefix}/config" , remove_folder_prefix = f"{folder_prefix}")
uncompress_zip_to_folder(zip_path = "./builtin/config/config.zip", extract_to= "./builtin")

## Definition of deployment functions

In [ ]:
def run_fab_command( command, capture_output: bool = False, silently_continue: bool = False, raw_output: bool = False):
    result = subprocess.run(["fab", "-c", command], capture_output=capture_output, text=True)
    if (not(silently_continue) and (result.returncode > 0 or result.stderr)):
       raise Exception(f"Error running fab command. exit_code: '{result.returncode}'; stderr: '{result}'")    
    if (capture_output and not raw_output): 
        output = result.stdout.strip()
        return output
    elif (capture_output and raw_output):
        return result

def fab_get_workspace_id(name):
    result = run_fab_command(f"get /{name} -q id" , capture_output = True, silently_continue= True)
    return result

def fab_workspace_exists(name):
    id = run_fab_command(f"get /{name} -q id" , capture_output = True, silently_continue= True)
    return(id)

def fab_delete_workspace(name):
    result = subprocess.run(["fab", "-c", "ls " + str(name)], capture_output=True, text=True)
    for item in result.stdout.split("\n"):
        x= subprocess.run(["fab", "-c", " del " + str(name)+"/"+str(item) + " -f"], capture_output=True, text=True)
        print(x)

def fab_get_id(name):
    id = run_fab_command(f"get /{trg_workspace_name}/{name} -q id" , capture_output = True, silently_continue= True)
    return(id)

def fab_get_item(name):
    item = run_fab_command(f"get /{trg_workspace_name}/{name}" , capture_output = True, silently_continue= True)
    return(item)

def fab_get_eventstreamConnectionString(itemId):
    item = run_fab_command(f"api -X get /workspaces/{trg_workspace_id}/eventstreams/{itemId}/topology" , capture_output = True, silently_continue= True)
    topologyEventstream=json.loads(item)
    SourceID=topologyEventstream["text"]["sources"][0]["id"]
    connection = run_fab_command(f"api -X get /workspaces/{trg_workspace_id}/eventstreams/{itemId}/sources/{SourceID}/connection" , capture_output = True, silently_continue= True)
    connection = json.loads(connection)
    return(connection["text"]["accessKeys"]["primaryConnectionString"])

def fab_get_display_name(name):
    display_name = run_fab_command(f"get /{trg_workspace_name}/{name} -q displayName" , capture_output = True, silently_continue= True)
    return(display_name)

def fab_get_kusto_query_uri(name):
    connection = run_fab_command(f"get /{trg_workspace_name}/{name} -q properties.queryServiceUri -f", capture_output = True, silently_continue= True)
    connection = connection.split("\n")[1]
    return(connection)

def fab_get_kusto_ingest_uri(name):
    connection = run_fab_command(f"get /{trg_workspace_name}/{name} -q properties.ingestionServiceUri -f", capture_output = True, silently_continue= True)
    connection = connection.split("\n")[1]
    return(connection)

def fab_get_folders():
    response = run_fab_command(f"api workspaces/{trg_workspace_id}/folders", capture_output = True, silently_continue= True)
    return(json.loads(response).get('text',{}).get('value',[]))

def fab_get_folder(folder_name):
    for f in fab_get_folders():
        if f.get('displayName') == folder_name:
            return f
    return None

def fab_assign_item_folder(name,folder):
    folder_details = fab_get_folder(folder)
    item_id = fab_get_id(name)

    if folder_details is None:
        payload = json.dumps({"displayName": folder})
        folder_details = run_fab_command(f"api -X post workspaces/{trg_workspace_id}/folders -i {payload}", capture_output = True, silently_continue= True)
        folder_details = json.loads(folder_details).get('text',{})

    payload = json.dumps({"folder": folder_details.get('id')})

    return run_fab_command(f"api -X patch workspaces/{trg_workspace_id}/items/{item_id} -i {payload}", capture_output = True, silently_continue= True)

def fab_add_schedule(name):
    item = run_fab_command(f"get /{trg_workspace_name}/{name} -q schedules" , capture_output = True, silently_continue= True)

    if len(json.loads(item)) == 0:
        schedule = get_schedule_by_name(name)

        return run_fab_command(f"job run-sch /{trg_workspace_name}/{name} -i {json.dumps(schedule)}" , capture_output = True, silently_continue=True)

    return f"""Job schedule for '{name}' already exists...
* Job schedule {item}""" 

def get_id_by_name(name):
    for it in deployment_order:
        if it.get("name") == name:
                return it.get("id")
    return None

def get_schedule_by_name(name):
    for it in deployment_order:
        if it.get("name") == name:
                return it.get("schedule")
    return None

def copy_to_tmp(name,child=None,type=None):
    child_path = "" if child is None else f".children/{child}/"
    type_path = "" if type is None else f"{type}/"
    shutil.rmtree("./builtin/tmp",  ignore_errors=True)
    path2zip = "./builtin/src/src.zip"
    with  ZipFile(path2zip) as archive:
        for file in archive.namelist():
            if file.startswith(f'src/{type_path}{name}/{child_path}'):
                archive.extract(file, './builtin/tmp')
    return(f"./builtin/tmp/src/{type_path}{name}/{child_path}" )

def get_mapping_table_new_from_type(type):
    result = ""
    filtered_data = list(filter(lambda item: item['Type'] == type, mapping_table))
    if len(filtered_data) > 0:
        result=filtered_data[0]["new"]
    return result

def get_mapping_table_new_from_old(old):
    result = ""
    filtered_data = list(filter(lambda item: item['old'] == old, mapping_table))
    if len(filtered_data) > 0:
        result=filtered_data[0]["new"]
    return result

def get_mapping_table_new_from_type_item(type,item):
    result = ""
    filtered_data = list(filter(lambda table: table["Type"] == type and table["Item"] == item, mapping_table))
    if len(filtered_data) > 0:
        result=filtered_data[0]["new"]
    return result

def get_mapping_table_parent_type(type,item,parent_type):
    parent_item = get_mapping_table_new_from_type_item(type,item)
    result = get_mapping_table_new_from_type_item(parent_type,parent_item)
    return result

def replace_ids_in_folder(folder_path, mapping_table):
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb', '.py', '.tmdl')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    for mapping in mapping_table:  
                        content = content.replace(mapping["old"], mapping["new"])
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)

def replace_kqldb_parent_eventhouse(folder_parth,parent_eventhouse):
    property_file = f"{folder_parth}/DatabaseProperties.json"
    with open(property_file, 'r', encoding='utf-8') as file:
        content = json.load(file)
        content["parentEventhouseItemId"] = fab_get_id(parent_eventhouse)
    with open(property_file, 'w', encoding='utf-8') as file:
        json.dump(content,file,indent=4)

def replace_eventstream_destination(folder_parth,it_destinations):
    property_file = f"{folder_parth}/eventstream.json"
    with open(property_file, "r", encoding="utf-8") as file:
        content = json.load(file)
        destinations = content.get("destinations",[])
        for destination in destinations:
            if destination.get("type") != "CustomEndpoint":
                filtered_data = list(filter(lambda table: table["name"] == destination.get("name") and table["type"] == destination.get("type"), it_destinations))
                if len(filtered_data) > 0:        
                    destination["properties"]["workspaceId"] = get_mapping_table_new_from_type_item("Workspace Id",trg_workspace_name)
                    destination["properties"]["itemId"] = get_mapping_table_new_from_type_item("KQL DB ID",filtered_data[0].get("itemName"))
                    if destination.get("properties",{}).get("databaseName") is not None:
                        destination["properties"]["databaseName"] = get_mapping_table_new_from_type_item("KQL DB Name",filtered_data[0].get("itemName"))
    with open(property_file, 'w', encoding='utf-8') as file:
        json.dump(content,file,indent=4)

def replace_kqldashboard_destination(folder_parth,it_datasources):
    property_file = f"{folder_parth}/RealTimeDashboard.json"
    with open(property_file, "r", encoding="utf-8") as file:
        content = json.load(file)
        datasources = content.get("dataSources",[])
        for datasource in datasources:
            filtered_data = list(filter(lambda table: table["name"] == datasource.get("name"), it_datasources))
            if len(filtered_data) > 0:        
                datasource["workspace"] = get_mapping_table_new_from_type_item("Workspace Id",trg_workspace_name)
                datasource["database"] = get_mapping_table_new_from_type_item("KQL DB ID",filtered_data[0].get("itemName"))
                datasource["clusterUri"] = get_mapping_table_parent_type("KQL DB Eventhouse",filtered_data[0].get("itemName"),"Kusto Query Uri")
    with open(property_file, 'w', encoding='utf-8') as file:
        json.dump(content,file,indent=4)

def deploy_item(name,child=None,it=None):
    parent = ""
    cli_parameter = ""

    # Copy and replace IDs in the item
    tmp_path = copy_to_tmp(name,child,it.get("type"))
    
    if child is not None:
        parent = name
        name = child     

    if ".KQLDatabase" in name:
        if child is not None:
            parent = parent if eventhouse_name == "" or eventhouse_name is None else f"{eventhouse_name}.Eventhouse"
        if it["parent"] is not None:
            parent = it["parent"] if eventhouse_name == "" or eventhouse_name is None else f"{eventhouse_name}.Eventhouse"
        mapping_table.append({"Type": "KQL DB Eventhouse", "Item": name, "old": it["parent"], "new": parent })  
        replace_kqldb_parent_eventhouse(tmp_path,parent)
    elif ".Eventhouse" in name:
        name = name if eventhouse_name == "" or eventhouse_name is None else f"{eventhouse_name}.Eventhouse"
    elif ".Eventstream" in name:
        replace_eventstream_destination(tmp_path,it["destinations"]) 
    elif ".Notebook" in name:
        cli_parameter = cli_parameter + " --format .py"
        replace_ids_in_folder(tmp_path, mapping_table)  
    elif ".DataPipeline" in name:
        replace_ids_in_folder(tmp_path, mapping_table)  
    elif ".Report" in name:
        replace_ids_in_folder(tmp_path, mapping_table)  
    elif ".SemanticModel" in name:
        replace_ids_in_folder(tmp_path, mapping_table)
    elif ".KQLDashboard" in name:
        replace_kqldashboard_destination(tmp_path, it["datasources"])

    print("")
    print("#############################################")
    print(f"Deploying {name}")      
    
    run_fab_command(f"import  /{trg_workspace_name}/{name} -i {tmp_path} -f {cli_parameter} ", silently_continue= True)

    new_id= fab_get_id(name)

    if ".KQLDatabase" in name:
        mapping_table.append({"Type": "KQL DB ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".Eventhouse" in name:
        query_uri = fab_get_kusto_query_uri(name)
        ingest_uri = fab_get_kusto_ingest_uri(name)
        mapping_table.append({"Type": "Kusto Query Uri", "Item": name, "old": it["kustoQueryUri"], "new": query_uri })        
        mapping_table.append({"Type": "Kusto Ingest Uri", "Item": name, "old": it["kustoIngestUri"], "new": ingest_uri })
        mapping_table.append({"Type": "Eventhouse ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".Eventstream" in name:
        if it.get("ConnectionStringCustomEndpoint") is not None:
            mapping_table.append({"Type": "Connection String Evenstream", "Item": name, "old": it["ConnectionStringCustomEndpoint"], "new": fab_get_eventstreamConnectionString(new_id) })
        mapping_table.append({"Type": "Eventstream ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".Notebook" in name:
        mapping_table.append({"Type": "Notebook ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".DataPipeline" in name:
        mapping_table.append({"Type": "Pipeline ID", "Item": name, "old": it["id"], "new": new_id })
        print("!DO NOT FORGET TO UPDATE TRIGGER OF THE PIPELINE!")
    elif ".Report" in name:
        mapping_table.append({"Type": "Report ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".SemanticModel" in name:
        mapping_table.append({"Type": "Semantic Model ID", "Item": name, "old": it["id"], "new": new_id })
    elif ".KQLDashboard" in name:
        mapping_table.append({"Type": "KQLDashboard ID", "Item": name, "old": it["id"], "new": new_id })

## CLI Login

In [ ]:
# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token

## Create or get current Workspace

In [ ]:
base_path = './builtin/'

deploy_order_path = os.path.join(base_path, 'config/deployment_order.json')
with open(deploy_order_path, 'r') as file:
        deployment_order = json.load(file)

#deploy workspace idempotent
if "NotFound" in fab_workspace_exists(f"{workspace_name}.Workspace"):
    run_fab_command(f"mkdir {workspace_name}.Workspace -P capacityname={capacity_name}.Capacity")
    print(f"New Workspace Create")

src_workspace_name = "Workspace.src"
src_workspace_id = get_id_by_name(src_workspace_name)

trg_workspace_id = fab_get_workspace_id(f"{workspace_name}.Workspace")
trg_workspace_name = f"{workspace_name}.Workspace"

print(f"Target Workspace Id: {trg_workspace_id}")
print(f"Target Workspace Name: {trg_workspace_name}")

mapping_table=[]

mapping_table.append({"Type": "Workspace Id", "Item": trg_workspace_name, "old": get_id_by_name(src_workspace_name), "new": trg_workspace_id })
mapping_table.append({"Type": "Workspace Blank Id", "Item": trg_workspace_name, "old": "00000000-0000-0000-0000-000000000000", "new": trg_workspace_id })
mapping_table.append({"Type": "Workspace Name", "Item": trg_workspace_name, "old": src_workspace_name, "new": trg_workspace_name.replace(".Workspace", "") })

display(mapping_table)

## Deployment Logic
This part iterates through all the items, gets the respective source code, replaces all IDs dynamically and deploys the new item

In [ ]:
exclude = [src_workspace_name]

for it in deployment_order:

    new_id = None
    
    name = it["name"]
    type = it.get("type")

    if not INSTALL_CAPACITY_MODULE and type == "Capacity":
        continue
    elif not INSTALL_GATEWAY_MODULE and type == "Gateway":
        continue
    elif not INSTALL_ACTIVITY_EVENTS_MODULE and type == "ActivityEvents":
        continue
    elif not INSTALL_INVENTORY_MODULE and type == "Inventory":
        continue

    if name in exclude:
        continue

    if not FIRST_RUN and ".Eventstream" in name:
        continue

    deploy_item(name,None,it)

    for child in it.get("children",[]):
        child_name = child["name"]
        deploy_item(name,child_name,child)   


In [ ]:
display(mapping_table)